In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('GFM_data.csv', sep = '\t')

In [2]:
import tensorflow as tf
import string
import requests

In [3]:
import string
#!pip install nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import wordnet

"""#redo"""
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
REPLACE_IP_ADDRESS = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
def extract_entities(text):
    names = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                names.append(' '.join(c[0] for c in chunk.leaves()))
    new_text = text
    for name in names:
        if name in text:
            new_text = new_text.replace(name, 'NLP')
    return new_text

def clean_text(x):
    ## removing names
    x = extract_entities(x)
    ## normalizing text by stripping white space and lower casing
    x =  x.lower().strip()
    ## removing urls
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x)
    ## removing phone numbers
    x = re.sub('\([0-9]{3}\)\s*[0-9]{3}-[0-9]{4}','',x)
    ## strip all non alphanumeric things
    x = re.sub('\n',' ',x)
    x = re.sub("[^a-zA-Z0-9 #]",'',x)
    x = re.sub("\s+",' ',x)
    text = x.replace('\n', ' ').lower()# lowercase text
    text = REPLACE_IP_ADDRESS.sub('', text) # remove ip address
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text
    # originally:
    # text = ' '.join([w for w in text.split() if not w in STOPWORDS])# delete stopwords from text
    text = [w for w in text.split() if not w in STOPWORDS]# delete stopwords from text   
    return text
"""
def clean_data(txt_elmt):
    # txt_elmt is each element (each post) in the "text" array
    tokens = txt_elmt.split() # each word
    
    # nothing changes. except removing punctuation
    mytable = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(mytable) for word in tokens]
    return [word.lower() for word in tokens if word.isalpha()]
"""
#for i in text:
# try with text[0] first
"""
tokens = clean_data(text[0]) # list of tokens
length = 51 # learn initial 50 words, predict the next word
lines = [] # lst of all training sequences
sufficiency = 200_000

for i in range(len(tokens)):
    sequence = tokens[i: i + length] # every 50-word
    lines += [' '. join(sequence)]
    if i > sufficiency:
        break
"""

"\ntokens = clean_data(text[0]) # list of tokens\nlength = 51 # learn initial 50 words, predict the next word\nlines = [] # lst of all training sequences\nsufficiency = 200_000\n\nfor i in range(len(tokens)):\n    sequence = tokens[i: i + length] # every 50-word\n    lines += [' '. join(sequence)]\n    if i > sufficiency:\n        break\n"

In [24]:
no_nan = df.dropna()
text = no_nan.get('Text').to_numpy()
#text = ' '.join(text) # join all array element
new_text = clean_text(text[0])
tokens = new_text#clean_data(new_text)
length = 51 # learn initial 50 words, predict the next word
lines = [] # lst of all training sequences
sufficiency = 200_000
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > sufficiency:
        break

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LambdaCallback

In [26]:
tokenizer = Tokenizer(oov_token="<OOV>") # OOV adds new words outside the trained words
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines) # all words become numbers

In [27]:
"""
X = []
y = []
maxlen=50
for i in range(len(sequences)):
    X.append(sequences[i][:-1])
    y.append(sequences[-1])
"""
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [28]:
# one-hot encoding
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [29]:
sequence_length = len(X[0])
 
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = sequence_length)) # input dim - vocabsize, output dim = 50
# 100 hidden layers
model.add(LSTM(100, return_sequences=True)) # do twice, return_sequences=True
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            4100      
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 82)                8282      
Total params: 163,282
Trainable params: 163,282
Non-trainable params: 0
_________________________________________________________________


In [31]:
import random
import sys
import io
maxlen = 51
words = sorted(list(set(new_text)))
word_indices = dict((w, i) for i, w in enumerate(words))
indices_word = dict((i, w) for i, w in enumerate(words))

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1
                
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:] + next_word

            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()

In [32]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model.fit(X, y, batch_size=256, epochs=100, callbacks=[print_callback])

Epoch 1/100
1/1 [==============================] - 0s 57ms/step - loss: 4.3941 - accuracy: 0.1463

----- Generating text after Epoch: 0
----- diversity: 0.2
['----- Generating with seed: "I am dying of lymphoma. My wife, Laura, has been nothing but a hero during this time. She is about to lose two incomes (mine and hers) as she manages and does research in a lab we shared together.\xa0My life insurance is tiny thanks to academia and our savings are nearly nonexistent.\xa0My biggest fear is that she won\'t have resources to get up on her feet. Please consider supporting her through my absence."'
 '----- Generating with seed: "Mattress Mack invites you to help him SAVE CAMP HOPE by donating to this campaign today! The PTSD Foundation of America\'s Camp Hope is a facility located in Houston, TX that offers temporary housing at no cost to our nation\'s veterans and their families as veterans find help for combat related Post Traumatic Stress Disorder (PTSD). In addition to temporary housin

AttributeError: 'numpy.ndarray' object has no attribute 'decode'

In [ ]:
seed_text=lines[len(lines) - 1]
seq_length = X.shape[1]
seed_text

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []

    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
        if index == y_predict:
            predicted_word = word
            break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
    return ' '.join(text)

In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

https://kgptalkie.com/text-generation-using-tensorflow-keras-and-lstm/

https://www.youtube.com/watch?v=VAMKuRAh2nc&t=1607s&ab_channel=KGPTalkie

In [ ]:
a = ['hello', 'hi']
b=' '.join(a)
b